In [1]:
import numpy as np
import itertools
import galois
from sympy import Matrix
from scipy.special import comb, perm
from tqdm import *

In [2]:
def setmask(M, i, j, s):
        M[i][j] = M[j][i] = s

def graph():
    ''' 
        return: n: number of vertices.
                mask: n x n matrix . M[i,j] = -1 means no edge between i and j, M[i,j]=1 means the edge ij is labeled 1, M[i,j]=0 means the edge ij is labeled 0.
    '''
    n = 12
    mask = -np.ones((n,n))
    setmask(mask, 0, 1, 1) 
    setmask(mask, 0, 4, 0) 
    setmask(mask, 1, 2, 0) 
    setmask(mask, 1, 4, 0) 
    setmask(mask, 1, 5, 0) 
    setmask(mask, 2, 5, 1) 
    setmask(mask, 3, 4, 1) 
    setmask(mask, 3, 7, 0) 
    setmask(mask, 4, 5, 0) 
    setmask(mask, 4, 7, 0) 
    setmask(mask, 4, 8, 1) 
    setmask(mask, 5, 6, 1) 
    setmask(mask, 5, 8, 0) 
    setmask(mask, 5, 9, 0) 
    setmask(mask, 6, 9, 0) 
    setmask(mask, 7, 8, 1) 
    setmask(mask, 7, 10, 1) 
    setmask(mask, 8, 9, 0) 
    setmask(mask, 8, 10, 0) 
    setmask(mask, 8, 11, 0) 
    setmask(mask, 9, 11, 1)
    return n, mask


In [3]:
class Solver():
    def __init__(self, n, M, t) -> None:
        self.GF = galois.GF(2)
        self.n = n  # graph order
        self.M = M  # n x n matrix . M[i,j] = -1 means no edge between i and j, M[i,j]=1 means the edge ij is labeled 1, M[i,j]=0 means the edge ij is labeled 0.
        self.t = t  # gonna test whether diam(I(G)) <= t
        self.vectors = self.gen_vector(t)  # t * (2**t) matrix, the colomns are all vectors in F_2^t

    
    def gen_vector(self, kk):
        ''' 
            return a (2**t
        '''
        if kk == 1:
            return self.GF([[0,1]])
        else:
            L = self.gen_vector(kk-1)
            LL = np.vstack((self.GF(np.zeros((1, L.shape[1]), dtype=np.int8)), L))
            RR = np.vstack((self.GF(np.ones((1, L.shape[1]), dtype=np.int8)), L))
            return np.hstack((LL, RR))
    
    def check_LR(self, L1, L2, V1, V2):
        ''' 
            input: L1 and L2 are disjoint vertex sets with n_i vertices respectively.
                    Vi is a n_i x t matrix, with columns are vectors in F_2^t assigned to vertices in Li.
            return: True if the vector assignment of L1 \cup L2 is valid in G
        '''
        for i, j in itertools.product(range(L1.shape[0]), range(L2.shape[0])):
            if self.M[L1[i]][L2[j]] != -1 and np.dot(V1[:,i], V2[:,j]) != self.M[L1[i]][L2[j]]:
                return False
        return True

    def solve_rec(self, L=None, debug=True):
        ''' 
            input:  L: the ndarray of vertices
            return: If len(L) < self.n: # that is G[L] is a proper subgraph
                        return a list with each element is a len(L) * t matrix
                                whose columns representing len(L) vectors in F_2^t assigned to vertices in L which is valid ( inner product of u and v is label(uv) )

                    If len(L) == self.n: # that is, L contains all vertices in G
                        When finding a solution, return True and the vector assignment
                        If cannot find a solution, return False 

            Using divide and conquer method
        '''
        if L is None:
            L = np.arange(self.n)
        if L.shape[0] == 1:
            return [ self.vectors[:, i].reshape((-1,1)) for i in range(self.vectors.shape[1])]
        
        L1 = L[:int(L.shape[0]/2)]
        L2 = L[int(L.shape[0]/2):]

        R = []

        L1Result = self.solve_rec(L1, debug=debug)
        L2Result = self.solve_rec(L2, debug=debug)
        for idx1, v1 in enumerate(L1Result):
            for idx2, v2 in enumerate(L2Result):
                if debug is True:
                    print('\rCalculation {}, {}'.format(((idx1 * len(L2Result)) + idx2) / (len(L1Result) * len(L2Result)),L), end='')
                
                if self.check_LR(L1, L2, v1, v2):
                    if L.shape[0] == self.n:
                        print('\nSucceed')
                        return True, np.hstack((v1,v2))
                    else:
                        R.append(np.hstack((v1,v2)))

        if L.shape[0] == self.n:
            print('\nFail')
            return False
        else:
            return R
                            
        

In [4]:
n, M = graph()
sol = Solver(n, M, 3)

In [5]:
sol.solve_rec()
# cost about 26s

Calculation 0.9999945746527777, [ 0  1  2  3  4  5  6  7  8  9 10 11]]]]]
Fail


False

The solver failed means $diam(I(G)) > 3$.
By the upper bound from treewidth, we have $diam(I(G)) = 4$.
That is, we find a outer-planar graph of inversion diameter four. 